In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def create_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def close_popups(driver):
    time.sleep(3)
    try:
        driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
        print("✅ Cookie popup closed")
        time.sleep(1)
    except:
        print("ℹ️ No cookie popup")

    try:
        driver.find_element(By.XPATH, '//span[text()="Later"]').click()
        print("✅ 'Later' popup closed")
        time.sleep(1)
    except:
        print("ℹ️ No 'Later' popup")


def get_total_pages(driver):
    try:
        time.sleep(2)
        element = driver.find_element(By.XPATH, '//*[@id="stats-app-root"]/section/section/div[4]/div[2]/div/div/div[1]/div[7]/button/span')
        total_pages = int(element.text)
        print(f" 共 {total_pages} 頁")
        return total_pages
    except:
        print("⚠️ 無法取得總頁數，預設 1")
        return 1


def parse_table(driver):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table")

    if not table:
        return [], []

    thead = table.find("thead")
    headers = []
    for th in thead.find_all("th"):
        abbr = th.find("abbr")
        if abbr and abbr.string:
            headers.append(abbr.string.strip())
        elif abbr:
            headers.append(abbr.contents[0].strip() if abbr.contents else "")
        else:
            headers.append(th.text.strip())

    # 解析資料列
    tbody = table.find("tbody")
    data = {}
    # 從 tbody 開始處理每個 <tr>
    for row in tbody.find_all("tr"):

        # 找 <th>（第一格，是 PLAYER 名 + index）
        player_th = row.find("th")

        # 找該 <th> 裡的第一個 <div>（就是 index）
        index_div = player_th.find("div", class_="index-qx7H9R5s")
        index = index_div.text.strip() if index_div else ""

        a_tag = player_th.find("a")
        name = a_tag["aria-label"].strip() if a_tag and "aria-label" in a_tag.attrs else ""

        player_key = f"{index};{name}"

        # 抓該列所有 <td>
        cols = row.find_all("td")
        row_data = [td.text.strip() for td in cols]
        data[player_key] = row_data

    return headers, data
    
        
def click_expanded_button(driver):
    try:
        expanded_button = driver.find_element(
            By.XPATH, '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button/span'
        )
        expanded_button.click()
        print(" Expanded 按鈕已點擊，等待表格更新...")

        # wait for expanded specific column show up，ex: BABIP
        WebDriverWait(driver, 10).until(
            #EC.presence_of_element_located((By.XPATH, '//abbr[text()="BABIP"]'))   # get player
            EC.presence_of_element_located((By.XPATH, '//abbr[text()="GDP"]'))   # get team
        )
        print("✅ 已切換為 Expanded，資料已更新完成")
        time.sleep(3)   # column show up but <td> doesn't
    except Exception as e:
        print("❌ 無法切換 Expanded 或等待失敗：", e)


def remove_columns(headers, data_dict, columns_to_remove):
    for col in columns_to_remove:
        if col in headers:
            idx = headers.index(col)
            for k in data_dict:
                del data_dict[k][idx]
            del headers[idx]


### getting players info

In [ ]:
def scrape_mlb_pitching(year, filename, first_write=False):
    driver = create_driver(headless=False)
    base_url = f"https://www.mlb.com/stats/pitching/{year}?playerPool=ALL"

    # 開始進入第一頁，處理彈窗 + 取得頁數
    driver.get(base_url)
    close_popups(driver)
    total_pages = get_total_pages(driver)

    final_headers = []    
    for page in range(1, total_pages + 1):
        url = f"https://www.mlb.com/stats/pitching/{year}?page={page}&playerPool=ALL"
        print(f"🔄 正在抓取：{url}")
        start = time.time()
        driver.get(url)
        print(f"get {year}: takes {time.time() - start} s")

        time.sleep(2)

        # get standard data
        std_headers, std_data = parse_table(driver)
        time.sleep(1)
        # click Expanded
        click_expanded_button(driver)
        # get expanded data
        exp_headers, exp_data = parse_table(driver)
        # remove duplicated headers
        if "PLAYER" in exp_headers:
            exp_headers.remove("PLAYER")
        remove_columns(exp_headers, exp_data, ["TEAM"])

        # merge
        rows = []
        for key in std_data:
            std_row = std_data.get(key, [])
            exp_row = exp_data.get(key, [])
            full_row = [year, key] + std_row + exp_row
            rows.append(full_row)
        # header define
        if not final_headers:
            final_headers = ["Season Year"] + std_headers + exp_headers
            print(final_headers)
        # append to csv
        df = pd.DataFrame(rows, columns=final_headers)
        df.to_csv(filename, mode="a", index=False, header=first_write)
        first_write = False
        print(f"✅ 第 {page} 頁已寫入")

    driver.quit()

# get player pitching
if __name__ == "__main__":
    filename = "mlb_pitching_all_years.csv"
    first_write = True  # 第一筆寫 header

    for year in range(2003, 2024):
        scrape_mlb_pitching(year, filename, first_write)
        first_write = False  # 後面就不寫 header 了

✅ Cookie popup closed
ℹ️ No 'Later' popup
📄 共 25 頁
🔄 正在抓取：https://www.mlb.com/stats/pitching/2003?page=1&playerPool=ALL
get 2003: takes 1.138113021850586 s
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
['Season Year', 'PLAYER', 'TEAM', 'W', 'L', 'ERA', 'G', 'GS', 'CG', 'SHO', 'SV', 'SVO', 'IP', 'H', 'R', 'ER', 'HR', 'HB', 'BB', 'SO', 'WHIP', 'AVG', 'TBF', 'NP', 'P/IP', 'QS', 'GF', 'HLD', 'IBB', 'WP', 'BK', 'GDP', 'GO/AO', 'SO/9', 'BB/9', 'K/BB', 'BABIP', 'SB', 'CS', 'PK']
✅ 第 1 頁已寫入
🔄 正在抓取：https://www.mlb.com/stats/pitching/2003?page=2&playerPool=ALL
get 2003: takes 2.518733024597168 s
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 第 2 頁已寫入
🔄 正在抓取：https://www.mlb.com/stats/pitching/2003?page=3&playerPool=ALL
get 2003: takes 2.417022943496704 s


KeyboardInterrupt: 

In [39]:
# read csv & describe the data
filename = "final_data/mlb_pitching_all_years.csv"
df = pd.read_csv(filename, encoding="utf-8")
print(df.describe())
print(df.info())
print(df.head(5))
print(df.columns)
print(f"All: {len(df)} datas")

        Season Year             W             L             G            GS  \
count  15036.000000  15036.000000  15036.000000  15036.000000  15036.000000   
mean    2013.690077      3.291301      3.291301     26.597832      6.583001   
std        6.094008      4.105850      3.647177     21.541593     10.689683   
min     2003.000000      0.000000      0.000000      1.000000      0.000000   
25%     2008.000000      0.000000      0.000000      8.000000      0.000000   
50%     2014.000000      2.000000      2.000000     22.000000      0.000000   
75%     2019.000000      5.000000      5.000000     37.000000      9.000000   
max     2023.000000     24.000000     21.000000     94.000000     36.000000   

                 CG           SHO            SV           SVO            IP  \
count  15036.000000  15036.000000  15036.000000  15036.000000  15036.000000   
mean       0.151703      0.065709      1.660149      2.502527     58.426437   
std        0.614045      0.314708      6.233519    

### getting team info

In [4]:
def scrape_team_pitching(year, filename, first_write=False):
    url = f"https://www.mlb.com/stats/team/pitching/{year}"
    driver = create_driver(headless=True)
    driver.get(url)
    close_popups(driver)

    # get standard data
    std_headers, std_data = parse_table(driver)

    # modify key value to identify flash is ok 
    click_expanded_button(driver)
    exp_headers, exp_data = parse_table(driver)
    
    # remove duplicate columns
    if "TEAM" in exp_headers:
        exp_headers.remove("TEAM")
    remove_columns(exp_headers, exp_data, ["LEAGUE"])

    # merge date
    final_headers = ["Season Year"] + std_headers + exp_headers
    rows = []
    for key in std_data:
        std_row = std_data.get(key, [])
        exp_row = exp_data.get(key, [])
        row = [year, key] + std_row + exp_row
        rows.append(row)

    # append in csv file
    df = pd.DataFrame(rows, columns=final_headers)
    df.to_csv(filename, mode="a", index=False, header=first_write)
    print(f"✅ {year} 年資料完成寫入！")
    driver.quit()

if __name__ == "__main__":
    filename = "mlb_team_pitching_all_years.csv"
    first_write = True

    for year in range(2003, 2024):
        scrape_team_pitching(year, filename, first_write)
        first_write = False  # 後面都不要再寫欄位名稱


✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2003 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2004 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2005 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2006 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2007 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2008 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2009 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2010 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 'Later' popup
 Expanded 按鈕已點擊，等待表格更新...
✅ 已切換為 Expanded，資料已更新完成
✅ 2011 年資料完成寫入！
✅ Cookie popup closed
ℹ️ No 

In [38]:
# read csv & describe the data
filename = "final_data/mlb_team_pitching_all_years.csv"
df = pd.read_csv(filename, encoding="utf-8")
print(df.describe())
print(df.info())
print(df.head(5))
print(df.columns)
print(f"All: {len(df)} datas")

       Season Year           W           L         ERA           G  \
count   630.000000  630.000000  630.000000  630.000000  630.000000   
mean   2013.000000   78.552381   78.552381    4.215175  157.114286   
std       6.060112   16.038099   16.015482    0.542755   21.764448   
min    2003.000000   19.000000   17.000000    2.800000   58.000000   
25%    2008.000000   71.000000   71.000000    3.830000  162.000000   
50%    2013.000000   80.500000   79.000000    4.175000  162.000000   
75%    2018.000000   90.000000   89.000000    4.590000  162.000000   
max    2023.000000  111.000000  119.000000    5.840000  163.000000   

               GS          CG         SHO          SV         SVO  ...  \
count  630.000000  630.000000  630.000000  630.000000  630.000000  ...   
mean   157.114286    3.620635    9.474603   39.622222   59.726984  ...   
std     21.764448    3.063797    4.111540    9.100576   11.581675  ...   
min     58.000000    0.000000    0.000000    6.000000   16.000000  ...   

### utils

In [ ]:
# # click year button  (get the first one)
# dropdowns = driver.find_elements(By.CLASS_NAME, "bui-dropdown__control")
# dropdowns[4].click() 
# time.sleep(2)

# # select year
# year_2023_option = driver.find_element(By.XPATH, '//div[text()="All Players"]')
# year_2023_option.click()
# time.sleep(2)